In [ ]:
# https://towardsdatascience.com/generating-modern-arts-using-generative-adversarial-network-gan-on-spell-39f67f83c7b4

### In order for our GAN model to work adequately, we must resize all the images that we will be feeding it to the same size. 128x128

In [33]:
import os
import numpy as np
from PIL import Image

# Define an image size and image channel
# Resize all to 128X128 and since coloured, channels is set to 3 (RBG)

IMAGE_SIZE = 128
IMAGE_CHANNELS = 3
IMAGE_DIR = 'Artsies/wikiart/Baroque/'

#Define the image dir path

image_path = IMAGE_DIR

training_data = []

# Iterating over the images inside the directory and resizing them using Pillow's resize method
print('resizing...')

# Pillow to reszie and appending them to a list as np array
for filename in os.listdir(image_path):
    path = os.path.join(image_path,filename)
    image = Image.open(path).resize((IMAGE_SIZE,IMAGE_SIZE), Image.ANTIALIAS)    
    training_data.append(np.asarray(image))
    
    
#Use np to reshape the array in a suitable formate and normalizing the data 
training_data = np.reshape(training_data, (-1, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS))
training_data = training_data / 127.5 -1

print('saving file...')
np.save('baroque_data.npy', training_data) # save the image array as npy binary file 
# Prevents us from going through all the images every time 

resizing...
saving file...


### Creating GAN - using Keras Deep

*Generatime Models* - Responsible for generating different kids of noise data
*Discriminative Models* - Responsible to discriminate whether the given data is real or fake 

- Generative models contantly trains itself to fool discriminative models by generating fak noise data
- Discriminative models trains itself from the training set to classify either the data is from dataset or not...and not to be fooled by generative models


### Loss Function

- Discrimator in GAN uses a cross entropy loss, since the discriminator's job is to classify

- In GAN, the discriminator is a binary classifier. It needs to classify the data as real or fake. 

### Code for GAN

In [34]:
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
import numpy as np
from PIL import Image
import os

In [37]:
# Defining some parameters

#Preview image Frame
PREVIEW_ROWS = 4
PREVIEW_COLS = 7 
PREVIEW_MARGIN = 4
SAVE_FREQ = 100

#Size vector to generate images fro
NOISE_SIZE = 100 # Latent dimension size to generate our images

#Configuration
EPOCHS = 1000 # number of iterations - defines how many times we want to iterate over our training images
BATCH_SIZE = 32 #Number of images to feed in every iteration

GENERATE_RES = 3
IMAGE_SIZE = 128 #Rows/cols - size of image 128X128

IMAGE_CHANNELS = 3 #The number of channels in our images - 3

# Note - images should always be of square size

In [38]:
training_data = np.load('Artsies/baroque_data.npy')
# To load file - using np's load function and passing file path as param
# Since data file in the root directory - no additional path param are required...otherwise:
# training_data = np.load(os.path.join('dirname','filename.npy'))

### Now create the Generator and Discriminator functions

But in simple language, here we are defining a convolutional layer which has a filter of size 3X3 and that filter strides over our image data. We have padding of same which means, no additional paddings are added. It remains the same as the original.

In [39]:
def build_discriminator(image_shape):
    
    model = Sequential() # Helps is create linear stacks of layers
    
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=image_shape, padding='same'))
    #Convolutional layer of 32 shape having kernel size of 4 and stride of 2 and padding same.
    # Since it is the first layer, it holds input_shape
    
    # We are defining a convolutional layer which has a filter of size 3X3 and that filter strides over
    # our image data...padding same which means, no additional paddings are added - remains same as original
    model.add(LeakyReLU(alpha=0.2)) #LeakyReLU is activation func
    model.add(Dropout(0.25)) #Dropouts & Batch normalization to prevent overfitting (same as below)
    
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(256, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
 
    model.add(Conv2D(512, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    #Since the discriminator's job is to classify whether a given image is fake or not, 
    # it is a binary classification task and sigmoid is activation that squeezes every value between 0 and 1
    model.add(Dense(1, activation='sigmoid')) #Las layer, fully connected layer with activation func sigmoid
    
    input_image = Input(shape=image_shape)
    validity = model(input_image)
    return Model(input_image, validity)

In [40]:
def build_generator(noise_size, channels):
    model = Sequential()
    model.add(Dense(4 * 4 * 256, activation='relu', input_dim=noise_size))
    #Since our generator model generates images from noise vector, our first layer is a fully connected Dense
    # layer of size 4096 (4*4*256)
    model.add(Reshape((4,4,256))) #Use Rehsape layer to reshape fully connect layer into shape 4X4X256
    
    #Layer blocks after this are just a Convolutional Layer with batch normalizations and activation func relu
    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    
    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    
    for i in range(GENERATE_RES):
        model.add(UpSampling2D())
        model.add(Conv2D(256,kernel_size=3,padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation('relu'))
        
    model.summary()
    model.add(Conv2D(channels, kernel_size=3, padding='same'))
    model.add(Activation('tanh'))
    
    input_ = Input(shape=(noise_size))
    generated_image = model(input_)
    
    return Model(input_, generated_image)

### Helper function to save the image after some iteration

In [44]:
def save_images(cnt,noise):
    image_array = np.full((
        PREVIEW_MARGIN + (PREVIEW_ROWS * (IMAGE_SIZE + PREVIEW_MARGIN)),
        PREVIEW_MARGIN + (PREVIEW_COLS * (IMAGE_SIZE + PREVIEW_MARGIN)), 3),
        255, dtype=np.uint8)
    
    generated_images = generator.predict(noise)
    
    generated_images = 0.5 * generated_images + 0.5
    
    image_count = 0
    for row in range(PREVIEW_ROWS):
        for col in range(PREVIEW_COLS):
            r = row * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            c = col * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            image_array[r:r + IMAGE_SIZE, c:c + IMAGE_SIZE] = generated_images[image_count] * 255
            image_count += 1
            
    output_path = 'output'
    if not os.path.exists(output_path):
        os.makedirs(output_path)
            
    filename = os.path.join(output_path,f"trained-{cnt}.png")
    im = Image.fromarray(image_array)
    im.save(filename)

#Inside the function, it generates frames from the parameters definied above and stores our generated image array 
# generated from noise input

### Compile Models and train them

In [42]:
image_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)

optimizer = Adam(1.5e-4, 0.5)

discriminator = build_discriminator(image_shape)
discriminator.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
generator = build_generator(NOISE_SIZE, IMAGE_CHANNELS)

random_input = Input(shape=(NOISE_SIZE,))

generated_image = generator(random_input)

discriminator.trainable = False

validity = discriminator(generated_image)

combined = Model(random_input, validity)
combined.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])

y_real = np.ones((BATCH_SIZE,1)) #notice the ones for real and 0s for fake
y_fake = np.zeros((BATCH_SIZE,1))

fixed_noise = np.random.normal(0,1,(PREVIEW_ROWS * PREVIEW_COLS, NOISE_SIZE))

cnt = 1
for epoch in range(EPOCHS):
    idx = np.random.randint(0,training_data.shape[0],BATCH_SIZE)
    x_real = training_data[idx]
    
    noise = np.random.normal(0,1,(BATCH_SIZE,NOISE_SIZE))
    x_fake = generator.predict(noise)
    
    discriminator_metric_real = discriminator.train_on_batch(x_real,y_real)
    
    discriminator_metric_generated = discriminator.train_on_batch(x_fake, y_fake)
    
    discriminator_metric = 0.5 * np.add(discriminator_metric_real,discriminator_metric_generated)
    
    generator_metric = combined.train_on_batch(noise, y_real)
    
if epoch % SAVE_FREQ == 0:
    save_images(cnt, fixed_noise)
    cnt += 1
    print(f"{epoch} epoch, Discriminator accuracy:{100*discriminator_metric[1]}, Generator accuracy:{100*generator_metric[1]}")
    
    
    
    
    

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 4096)              413696    
_________________________________________________________________
reshape_2 (Reshape)          (None, 4, 4, 256)         0         
_________________________________________________________________
up_sampling2d_6 (UpSampling2 (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 8, 8, 256)         590080    
_________________________________________________________________
batch_normalization_17 (Batc (None, 8, 8, 256)         1024      
_________________________________________________________________
activation_6 (Activation)    (None, 8, 8, 256)         0         
_________________________________________________________________
up_sampling2d_7 (UpSampling2 (None, 16, 16, 256)      

In [49]:
combined
